# Geração da Base Reduzida – Projeto PAR 2025

Este documento descreve o processo utilizado para **montagem, limpeza, alinhamento e preparação** da base reduzida utilizada nos experimentos do projeto PAR 2025. Toda a execução foi realizada em Google Colab, com integração ao Google Drive.

---

## 1. Download da Base Original

A primeira etapa consiste em baixar os arquivos disponibilizados por um dos integrantes do grupo via Google Drive.

São baixados:

* O arquivo ZIP com as imagens da base de treino (`training_set.zip`)
* O arquivo `.txt` contendo as anotações (`training_set.txt`)

O download foi realizado com `gdown` apontando para os IDs dos arquivos no Drive.

---

## 2. Extração do ZIP

Após o download, o arquivo ZIP é extraído para um diretório específico dentro do `/content`, permitindo acesso às imagens originais.

Essa extração é feita em duas etapas:

1. Extração inicial para `training_set/`
2. Extração organizada para `training_set_extracted/`, utilizada como base final

---

## 3. Carregamento e Construção do DataFrame de Labels

O arquivo `training_set.txt` contém as colunas:

* `filename`
* `top_color`
* `bottom_color`
* `gender`
* `hat`
* `bag`

Ele é lido e associado a caminhos completos de arquivo, permitindo verificação e processamento das imagens.

---

## 4. Correção de Imagens Problemáticas

Foram identificadas imagens com canais incorretos ou formato inconsistente. Essas imagens foram corrigidas manualmente utilizando Pillow, garantindo que todas passem a estar em RGB.

As imagens corrigidas foram sobrescritas diretamente no dataset.

---

## 5. Corte da Base até a Última Imagem Válida

Para evitar divergências entre quantidade de imagens e linhas do CSV, o DataFrame final é cortado até a **última imagem existente na pasta**.

O resultado é salvo como:

* `training_set_final.txt`

Este arquivo é a versão oficial utilizada para formar a base reduzida.

---

## Balanceamento da Base com Augmentation

### 6. Identificação de Classes Minoritárias
- As colunas `top_color` e `bottom_color` foram analisadas.
- Foram calculados percentis baixos para identificar:
  - **Classes extremamente baixas**
  - **Classes baixas**
- Apenas essas classes receberiam augmentation.

### 7. Definição dos Fatores de Augmentation
- Classes extremamente baixas: até **16×**
- Classes baixas: até **10×**
- Fatores calculados de forma conservadora para evitar oversampling pesado.

### 8. Técnicas de Augmentation Utilizadas
- Flip horizontal  
- Gaussian Noise  
- ISO Noise  
- Motion Blur  
- Combinação (Blur + Noise)

Cada técnica gera novas imagens artificiais nomeadas automaticamente.

### 9. Geração da Nova Base Balanceada
- Para cada imagem minoritária, diversas versões são geradas.
- As entradas correspondentes são adicionadas a um novo dataframe.

### 10. Visualização das Distribuições
Foram gerados gráficos de barras para:

- `top_color`
- `bottom_color`
- `gender`
- `hat`
- `bag`



In [ ]:
#PUXA DO DRIVE DO ARTHUR
!pip install -q gdown

training_zip_id = "1RIL4aU_LP6VwYbBKl7cS9XM7w1EqJEge"
training_txt_id = "1KsHdzNQBJmc6rWbB1wApOWD2krcuQN9L"



# Download
!gdown --id $training_zip_id -O training_set.zip
!gdown --id $training_txt_id -O training_set.txt

# Extrair zip
import zipfile

with zipfile.ZipFile("training_set.zip", 'r') as zip_ref:
    zip_ref.extractall("training_set")

print("Arquivos baixados e extraídos com sucesso!")


/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1RIL4aU_LP6VwYbBKl7cS9XM7w1EqJEge
From (redirected): https://drive.google.com/uc?id=1RIL4aU_LP6VwYbBKl7cS9XM7w1EqJEge&confirm=t&uuid=38a04426-848e-486e-b50b-50f82b18678c
To: /content/training_set.zip
100% 188M/188M [00:02<00:00, 88.5MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1KsHdzNQBJmc6rWbB1wApOWD2krcuQN9L
To: /content/training_set.txt
100% 6.39M/6.39M [00:00<00:00, 26.7MB/s]
Arquivos baixados e extraídos com sucesso!


In [ ]:
#EXTRAI ZIP
import pandas as pd
import os
import zipfile

# Configurações
zip_path = "/content/training_set.zip"
extract_dir = "/content/training_set_extracted"

# Criar pasta de extração
os.makedirs(extract_dir, exist_ok=True)

# Extrair ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("ZIP extraído para", extract_dir)

# Ler CSV de labels
labels_df = pd.read_csv("training_set.txt", sep=",", header=None)
labels_df.columns = ["filename", "top_color", "bottom_color", "gender", "hat", "bag"]

IMG_DIR = os.path.join(extract_dir, "training_set")
labels_df["filepath"] = labels_df["filename"].apply(lambda x: os.path.join(IMG_DIR, x))

# Conferir
print(labels_df.head())


ZIP extraído para /content/training_set_extracted
  filename  top_color  bottom_color  gender  hat  bag  \
0    1.jpg          1             1       0    0    0   
1    2.jpg          7             1       1    0    0   
2    3.jpg          4             2       0    0    0   
3    4.jpg          1             1       0    0    0   
4    5.jpg          1             1       0    0    0   

                                            filepath  
0  /content/training_set_extracted/training_set/1...  
1  /content/training_set_extracted/training_set/2...  
2  /content/training_set_extracted/training_set/3...  
3  /content/training_set_extracted/training_set/4...  
4  /content/training_set_extracted/training_set/5...  


In [ ]:
#CORRIGE IMAGENS PROBLEMATICAS
from PIL import Image

problem_imgs = ["000_45_1.jpg", "000_45_2.jpg"]
for img_name in problem_imgs:
    img_path = os.path.join(IMG_DIR, img_name)
    im = Image.open(img_path)
    rgb_im = im.convert('RGB')  # garante 3 canais
    rgb_im.save(img_path, "JPEG")  # sobrescreve como JPEG


In [ ]:
import pandas as pd
import os
import re

# Caminho da pasta de imagens
IMG_DIR = "/content/training_set/training_set"

# Ler CSV original
labels_df = pd.read_csv("training_set.txt", sep=",", header=None)
labels_df.columns = ["filename", "top_color", "bottom_color", "gender", "hat", "bag"]

# Função para ordenação alfa-numérica
def sorted_nicely(file_list):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(file_list, key=alphanum_key)

# Listar arquivos existentes na pasta
existing_files = os.listdir(IMG_DIR)
existing_files_sorted = sorted_nicely(existing_files)

# Filtrar CSV para imagens existentes
labels_df = labels_df[labels_df["filename"].isin(existing_files_sorted)].reset_index(drop=True)

# Ordenar CSV alfa-numérico
sorted_filenames = sorted_nicely(labels_df["filename"].tolist())
labels_df = labels_df.set_index("filename").loc[sorted_filenames].reset_index()

# Salvar CSV alinhado antes de cortar
labels_df.to_csv("training_set_aligned.txt", index=False)
print("TXT alinhado salvo como 'training_set_aligned.txt'")

# Encontrar a última imagem da pasta
last_image_name = existing_files_sorted[-1]

# Cortar o CSV até a última imagem
last_idx = labels_df[labels_df["filename"] == last_image_name].index[0]
labels_df_final = labels_df.iloc[:last_idx+1].reset_index(drop=True)

# Salvar CSV final cortado
labels_df_final.to_csv("training_set_final.txt", index=False)
print("TXT final cortado salvo como 'training_set_final.txt', até a última imagem:", last_image_name)

# Conferir
print(labels_df_final.tail(5))


TXT alinhado salvo como 'training_set_aligned.txt'
TXT final cortado salvo como 'training_set_final.txt', até a última imagem: VideoX_412_135_3.jpg
                   filename  top_color  bottom_color  gender  hat  bag
22284  VideoX_412_134_1.jpg         10             4       0    0    1
22285  VideoX_412_134_2.jpg          4             2       0    1    0
22286  VideoX_412_135_1.jpg         11             4       0    0    1
22287  VideoX_412_135_2.jpg          4             2       0    1    0
22288  VideoX_412_135_3.jpg          4             2       0    1    1


In [ ]:
!pip install albumentations


In [ ]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import albumentations as A
import random

# Definir seed para reprodutibilidade
random.seed(42)
np.random.seed(42)

# Diretório e leitura do dataset
IMG_DIR = "/content/training_set/training_set"
labels_df = pd.read_csv("training_set_final.txt", sep=",")
labels_df.columns = ["filename", "top_color", "bottom_color", "gender", "hat", "bag"]

print("=== IDENTIFICANDO APENAS AS CLASSES MAIS BAIXAS ===")

# Analisar distribuição das classes
top_counts = labels_df['top_color'].value_counts().sort_index()
bottom_counts = labels_df['bottom_color'].value_counts().sort_index()

print("Distribuição original TOP_COLOR:")
print(top_counts)
print("\nDistribuição original BOTTOM_COLOR:")
print(bottom_counts)

# Estratégia mais rigorosa - identificar apenas classes realmente baixas
# Usar percentis baixos para identificar classes minoritárias
top_10th_percentile = np.percentile(top_counts.values, 10)  # 10% menores
top_20th_percentile = np.percentile(top_counts.values, 20)  # 20% menores

bottom_10th_percentile = np.percentile(bottom_counts.values, 30)
bottom_20th_percentile = np.percentile(bottom_counts.values, 40)

# Classes extremamente baixas (bottom 10%)
VERY_LOW_TOP = top_counts[top_counts <= top_10th_percentile].index.tolist()
VERY_LOW_BOTTOM = bottom_counts[bottom_counts <= bottom_10th_percentile].index.tolist()

# Classes baixas (bottom 20% mas acima do 10%)
LOW_TOP = top_counts[(top_counts > top_10th_percentile) & (top_counts <= top_20th_percentile)].index.tolist()
LOW_BOTTOM = bottom_counts[(bottom_counts > bottom_10th_percentile) & (bottom_counts <= bottom_20th_percentile)].index.tolist()

print(f"\n=== CLASSES SELECIONADAS PARA AUGMENTATION ===")
print(f"TOP_COLOR - Extremamente baixas (≤{top_10th_percentile:.0f}): {VERY_LOW_TOP}")
print(f"Counts: {[top_counts[c] for c in VERY_LOW_TOP]}")
print(f"TOP_COLOR - Baixas ({top_10th_percentile:.0f} < x ≤ {top_20th_percentile:.0f}): {LOW_TOP}")
print(f"Counts: {[top_counts[c] for c in LOW_TOP]}")

print(f"\nBOTTOM_COLOR - Extremamente baixas (≤{bottom_10th_percentile:.0f}): {VERY_LOW_BOTTOM}")
print(f"Counts: {[bottom_counts[c] for c in VERY_LOW_BOTTOM]}")
print(f"BOTTOM_COLOR - Baixas ({bottom_10th_percentile:.0f} < x ≤ {bottom_20th_percentile:.0f}): {LOW_BOTTOM}")
print(f"Counts: {[bottom_counts[c] for c in LOW_BOTTOM]}")

# Função mais conservadora para augmentation
def calculate_conservative_augmentation(class_count, class_level, max_count_in_dataset):
    """Calcula augmentation de forma conservadora baseado no percentil"""
    if class_level == "very_low":
        # Para classes extremamente baixas: máximo 8x
        target_ratio = 0.30  # Tentar chegar a 15% da classe mais comum
        max_factor = 16
    elif class_level == "low":
        # Para classes baixas: máximo 5x
        target_ratio = 0.15  # Tentar chegar a 12% da classe mais comum
        max_factor = 10
    else:
        return 1  # Não aumentar outras classes

    target_count = int(max_count_in_dataset * target_ratio)
    if class_count >= target_count:
        return 1

    factor = min(max_factor, int(target_count / class_count))
    return max(1, factor)

# Definir transformações
augmentations = {
    'noise': A.AdditiveNoise(
        noise_type="gaussian",
        spatial_mode="shared",
        noise_params={"mean_range":[0,0],"std_range":[0.05,0.15]},
        approximation=1
    ),
    'blur': A.MotionBlur(
        blur_limit=7,
        p=1.0
    ),
    'ISONoise': A.ISONoise(
      color_shift=[0.01, 0.05],
      intensity=[0.1, 0.5]
  )
}

print(f"\nProcessando dataset com {len(labels_df)} imagens...")

# Lista para construir o novo dataset
new_dataset_rows = []
augmentation_stats = {"flip": 0, "noise": 0, "ISONoise": 0, "blur": 0, "combined": 0}

# Contadores para monitoramento
classes_processed = set()
images_augmented = 0

# Loop nas imagens
for index, row in labels_df.iterrows():
    # Sempre adicionar a imagem original primeiro
    new_dataset_rows.append(row)

    # Determinar se a imagem deve ser aumentada
    top_class = int(row["top_color"])
    bottom_class = int(row["bottom_color"])

    aug_factor = 1
    is_augmentable = False
    reason = []

    # Verificar apenas classes realmente baixas para top_color
    if top_class in VERY_LOW_TOP:
        top_factor = calculate_conservative_augmentation(
            top_counts[top_class], "very_low", top_counts.max()
        )
        aug_factor = max(aug_factor, top_factor)
        is_augmentable = True
        reason.append(f"top_very_low_{top_class}")
        classes_processed.add(f"top_{top_class}")
    elif top_class in LOW_TOP:
        top_factor = calculate_conservative_augmentation(
            top_counts[top_class], "low", top_counts.max()
        )
        aug_factor = max(aug_factor, top_factor)
        is_augmentable = True
        reason.append(f"top_low_{top_class}")
        classes_processed.add(f"top_{top_class}")

    # Verificar apenas classes realmente baixas para bottom_color
    if bottom_class in VERY_LOW_BOTTOM:
        bottom_factor = calculate_conservative_augmentation(
            bottom_counts[bottom_class], "very_low", bottom_counts.max()
        )
        aug_factor = max(aug_factor, bottom_factor)
        is_augmentable = True
        reason.append(f"bottom_very_low_{bottom_class}")
        classes_processed.add(f"bottom_{bottom_class}")
    elif bottom_class in LOW_BOTTOM:
        bottom_factor = calculate_conservative_augmentation(
            bottom_counts[bottom_class], "low", bottom_counts.max()
        )
        aug_factor = max(aug_factor, bottom_factor)
        is_augmentable = True
        reason.append(f"bottom_low_{bottom_class}")
        classes_processed.add(f"bottom_{bottom_class}")

    # Aplicar augmentation apenas se for classe realmente baixa
    if is_augmentable and aug_factor > 1:
        if images_augmented < 10:  # Log das primeiras 10 para debug
            print(f"Imagem {row['filename']}: {' + '.join(reason)} -> fator {aug_factor}")

        img_path = os.path.join(IMG_DIR, row["filename"])

        if not os.path.exists(img_path):
            print(f"Imagem não encontrada: {img_path}")
            continue

        img = cv2.imread(img_path)
        if img is None:
            print(f"Erro ao carregar imagem: {img_path}")
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        base_name = os.path.splitext(row["filename"])[0]
        ext = os.path.splitext(row["filename"])[1]

        # Lista de técnicas
        aug_techniques = ['flip', 'noise', 'ISONoise', 'blur', 'combined']

        # Criar augmentations
        for i in range(aug_factor - 1):
            technique = aug_techniques[i % len(aug_techniques)]

            if technique == 'flip':
                augmented_img = cv2.flip(img_rgb, 1)
                suffix = "flipped"

            elif technique == 'combined':
                # Combinar transformações para classes muito baixas
                combined_aug = A.Compose([
                    augmentations['blur'],
                    augmentations['noise']
                ])
                augmented = combined_aug(image=img_rgb)
                augmented_img = augmented["image"]
                suffix = "combined"

            else:
                # Aplicar transformação específica
                augmented = augmentations[technique](image=img_rgb)
                augmented_img = augmented["image"]
                suffix = technique

            # Gerar nome único
            aug_filename = f"{base_name}_{suffix}_{i+1}{ext}"
            aug_path = os.path.join(IMG_DIR, aug_filename)

            # Salvar imagem
            augmented_bgr = cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR)
            cv2.imwrite(aug_path, augmented_bgr)

            # Adicionar ao dataset
            aug_entry = row.copy()
            aug_entry["filename"] = aug_filename
            new_dataset_rows.append(aug_entry)

            # Atualizar estatísticas
            augmentation_stats[technique] += 1

        images_augmented += 1

    # Mostrar progresso
    if index % 500 == 0:
        print(f"Processadas {index} imagens... Augmentadas: {images_augmented}")

# Criar DataFrame final
augmented_labels_df = pd.DataFrame(new_dataset_rows)
augmented_labels_df.to_csv("training_set_balanced.txt", index=False)

print(f"\n=== PROCESSAMENTO CONCLUÍDO ===")
print(f"Classes processadas: {sorted(classes_processed)}")
print(f"Imagens que receberam augmentation: {images_augmented}")

augmented_count = len(augmented_labels_df) - len(labels_df)
print(f"Novas imagens criadas: {augmented_count}")
print(f"Dataset original: {len(labels_df)} imagens")
print(f"Dataset balanceado: {len(augmented_labels_df)} imagens")
print(f"Aumento total: {len(augmented_labels_df)/len(labels_df):.2f}x")

# Estatísticas detalhadas
print(f"\n=== TIPOS DE AUGMENTATION APLICADOS ===")
for aug_type, count in augmentation_stats.items():
    if count > 0:
        print(f"{aug_type.capitalize()}: {count}")

# Análise das distribuições finais - foco nas classes que foram aumentadas
new_top_counts = augmented_labels_df['top_color'].value_counts().sort_index()
new_bottom_counts = augmented_labels_df['bottom_color'].value_counts().sort_index()

print(f"\n=== VERIFICAÇÃO: CLASSES QUE FORAM AUMENTADAS ===")

if VERY_LOW_TOP or LOW_TOP:
    print("TOP_COLOR - Classes aumentadas:")
    for cls in VERY_LOW_TOP + LOW_TOP:
        antes = top_counts[cls]
        depois = new_top_counts[cls]
        print(f"  Classe {cls}: {antes} → {depois} ({depois/antes:.1f}x)")

if VERY_LOW_BOTTOM or LOW_BOTTOM:
    print("\nBOTTOM_COLOR - Classes aumentadas:")
    for cls in VERY_LOW_BOTTOM + LOW_BOTTOM:
        antes = bottom_counts[cls]
        depois = new_bottom_counts[cls]
        print(f"  Classe {cls}: {antes} → {depois} ({depois/antes:.1f}x)")

print(f"\nArquivo salvo como: training_set_balanced.txt")

In [ ]:
#DISTRIBUIÇÃO DA BASE
import subprocess, sys
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# Caminho do TXT
txt_path = "/content/training_set_balanced.txt"

print("Lendo arquivo TXT...\n")
df = pd.read_csv(txt_path, header=0, sep=",")
df.columns = ["filename", "top_color", "bottom_color", "gender", "hat", "bag"]

print("Arquivo carregado!")
print(f"Total de imagens: {len(df):,}\n")


# Função de contagem e plot
def contar_e_plotar(coluna, titulo):
    contagem = df[coluna].value_counts().sort_index()
    porcentagem = contagem / contagem.sum() * 100

    print(f"=== {titulo} ===")
    print(contagem)
    print(porcentagem.round(2))
    print()

    labels = contagem.index.astype(str)
    valores = contagem.values

    plt.figure(figsize=(8, 4))
    bars = plt.bar(labels, valores, color=plt.cm.tab10.colors)

    # Exibir valores e % acima das barras
    for i, (v, p) in enumerate(zip(valores, porcentagem)):
        plt.text(i, v + max(valores)*0.01, f"{v}\n({p:.1f}%)",
                 ha='center', va='bottom', fontsize=8)

    plt.title(f"Distribuição - {titulo}")
    plt.xlabel("Classe (número)")
    plt.ylabel("Quantidade")
    plt.tight_layout()
    plt.show()


# Gerar gráficos
contar_e_plotar("top_color", "Top Color (Parte Superior)")
contar_e_plotar("bottom_color", "Bottom Color (Parte Inferior)")
contar_e_plotar("gender", "Gênero")
contar_e_plotar("hat", "Uso de Chapéu")
contar_e_plotar("bag", "Uso de Bolsa")

print("Análise concluída!")


In [ ]:
from google.colab import files
import zipfile
import os

# Caminho da pasta com as imagens
img_folder = "/content/training_set/training_set"

# Caminho do arquivo de labels
labels_file = "/content/training_set_balanced.txt"

# Nome do zip final
zip_filename = "training_balanced.zip"

# Criar o ZIP
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Adicionar todas as imagens da pasta
    for root, dirs, files_in_dir in os.walk(img_folder):
        for file in files_in_dir:
            file_path = os.path.join(root, file)
            # Caminho relativo para dentro do ZIP
            arcname = os.path.relpath(file_path, os.path.dirname(img_folder))
            zipf.write(file_path, arcname)

    # Adicionar o arquivo de labels
    zipf.write(labels_file, os.path.basename(labels_file))

# Fazer download do ZIP
files.download(zip_filename)
